In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tushare as ts
import sqlite3
import yaml

from datetime import datetime as dt
from helpers.data_processing import (
    remove_ST_and_list_date,
    remove_non_tradable_stocks,
    save_to_db,
    read_from_db
)

In [3]:
sns.set_theme()

with open("../configs/environ.yml", "r") as rf:
    token = yaml.safe_load(rf)["token"]

pro = ts.pro_api(token)

In [4]:
today = dt.today().strftime("%Y%m%d")
hs300_start, hs300_end = "20240101", "20240131"
zz500_start, zz500_end = "20231201", "20231231"
db_filename = "data/stocks.db"
today

'20240131'

## 读取全部A股、沪深300和中证500

In [5]:
all_stocks = pro.stock_basic(ts_code="", list_status="L", fields="ts_code, symbol, name, list_status, list_date")
all_stocks

,ts_code,symbol,name,list_status,list_date
0,000001.SZ,000001,平安银行,L,19910403
1,000002.SZ,000002,万科A,L,19910129
2,000004.SZ,000004,国华网安,L,19910114
3,000005.SZ,000005,ST星源,L,19901210
4,000006.SZ,000006,深振业A,L,19920427
...,...,...,...,...,...
5342,873703.BJ,873703,广厦环能,L,20231205
5343,873726.BJ,873726,卓兆点胶,L,20231019
5344,873806.BJ,873806,云星宇,L,20240111
5345,873833.BJ,873833,美心翼申,L,20231108


In [6]:
trade_info = pro.daily("", trade_date=today)
trade_info.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,831834.BJ,20240130,5.90,6.03,5.71,5.99,5.90,0.09,1.5254,14196.31,8336.898
1,300655.SZ,20240130,7.67,7.75,7.38,7.41,7.67,-0.26,-3.3898,103407.29,78304.418
2,000931.SZ,20240130,5.18,5.20,5.01,5.03,5.25,-0.22,-4.1905,73566.76,37647.336
3,002136.SZ,20240130,10.36,10.36,9.84,9.87,10.34,-0.47,-4.5455,24603.00,24821.085
4,301314.SZ,20240130,35.37,35.98,33.99,34.11,36.00,-1.89,-5.2500,5237.80,18313.565


In [7]:
hs300 = pro.index_weight(index_code="399300.SZ", start_date=hs300_start, end_date=hs300_end)
hs300.rename(columns={"con_code": "ts_code"}, inplace=True)
hs300

,index_code,ts_code,trade_date,weight
0,399300.SZ,600519.SH,20240102,6.1717
1,399300.SZ,601318.SH,20240102,2.4772
2,399300.SZ,300750.SZ,20240102,2.4139
3,399300.SZ,600036.SH,20240102,1.9907
4,399300.SZ,000333.SZ,20240102,1.5644
...,...,...,...,...
295,399300.SZ,601808.SH,20240102,0.0512
296,399300.SZ,300979.SZ,20240102,0.0471
297,399300.SZ,000800.SZ,20240102,0.0458
298,399300.SZ,601059.SH,20240102,0.0333


In [8]:
zz500 = pro.index_weight(index_code="000905.SH", start_date=zz500_start, end_date=zz500_end)
zz500.rename(columns={"con_code": "ts_code"}, inplace=True)
zz500

,index_code,ts_code,trade_date,weight
0,000905.SH,300502.SZ,20231229,0.548
1,000905.SH,002422.SZ,20231229,0.537
2,000905.SH,002028.SZ,20231229,0.502
3,000905.SH,300418.SZ,20231229,0.498
4,000905.SH,600157.SH,20231229,0.477
...,...,...,...,...
495,000905.SH,688375.SH,20231229,0.045
496,000905.SH,603355.SH,20231229,0.040
497,000905.SH,603868.SH,20231229,0.038
498,000905.SH,688248.SH,20231229,0.033


## 剔除ST、上市不到一年的A股

In [9]:
all_stocks_out = remove_ST_and_list_date(all_stocks, min_list_duration=365)
all_stocks_out

,ts_code,symbol,name,list_status,list_date
0,000001.SZ,000001,平安银行,L,19910403
1,000002.SZ,000002,万科A,L,19910129
2,000004.SZ,000004,国华网安,L,19910114
4,000006.SZ,000006,深振业A,L,19920427
6,000008.SZ,000008,神州高铁,L,19920507
...,...,...,...,...,...
5331,873223.BJ,873223,荣亿精密,L,20220609
5332,873305.BJ,873305,九菱科技,L,20221221
5333,873339.BJ,873339,恒太照明,L,20221117
5334,873527.BJ,873527,夜光明,L,20221027


## 保留当日vol > 0的股票

In [10]:
all_stocks_out_final = remove_non_tradable_stocks(all_stocks_out, trade_info)
all_stocks_out_final.shape

(4910,)

In [11]:
hs300_final = remove_non_tradable_stocks(hs300, trade_info)
hs300_final.shape

(300,)

In [12]:
zz500_final = remove_non_tradable_stocks(zz500, trade_info)
zz500_final.shape

(500,)

## 保存数据到DB

In [13]:
table_names = {
    "A_share": all_stocks_out_final,
    "HS300": hs300_final,
    "ZZ500": zz500_final
}

In [14]:
for table_name, stock_series in table_names.items():
    save_to_db(today, stock_series, table_name, db_filename)

In [15]:
for table_name, stock_series in table_names.items():
    df_iter = read_from_db(table_name, db_filename)
    print(f"{table_name}:")
    print(df_iter)
    print("-" * 100)

A_share:
                      status  turnover_rate       pe      pb      total_mv  \
date       ts_code                                                           
2024-01-29 000001.SZ       1         1.2245   4.1356  0.4754  1.882374e+07   
           000002.SZ       1         1.1787   5.3066  0.4746  1.200229e+07   
           000004.SZ       1         9.2760      NaN  6.2164  1.972466e+05   
           000006.SZ       1         7.6998  15.2173  0.8214  6.385477e+05   
           000008.SZ       1         0.8795      NaN  1.4519  6.139014e+05   
...                      ...            ...      ...     ...           ...   
2024-01-30 873169.BJ       1            NaN      NaN     NaN           NaN   
           873223.BJ       1            NaN      NaN     NaN           NaN   
           873305.BJ       1            NaN      NaN     NaN           NaN   
           873339.BJ       1            NaN      NaN     NaN           NaN   
           873527.BJ       1            NaN      NaN   